In [1]:

from kafka import KafkaProducer, KafkaConsumer
from time import sleep
import json
from datetime import datetime
from confluent_kafka import Producer
import socket
import pymongo
from pymongo.errors import BulkWriteError

from pymongo import MongoClient, ASCENDING, DESCENDING

from pprint import pprint

from time import sleep
from bson.objectid import ObjectId 
import subprocess
from itertools import combinations, combinations_with_replacement
from EtudeBinance import Makethefile,getpairs

{'id': 787198238, 'price': '3007.38000000', 'qty': '0.54960000', 'quoteQty': '1652.85604800', 'time': 1647967172681, 'isBuyerMaker': True, 'isBestMatch': True}


In [2]:
symboles=list(filter(('').__ne__, getpairs()))
listesymboles=["BTC","ETH","USDT","USD","BNB","XRP","ADA","DOGE","LTC","BCH","SOL","ETC","LUNA","AVAX","SHIB","DOT","MATIC","AAVE","UNI","VRA","USDC","ADX","USDC","APE","JASMY","GALA","WAVES","LRC"]
temp = combinations(listesymboles, 2)
newlist=[]
for j in list(temp):
    newlist.append(j[0]+j[1])
    newlist.append(j[1]+j[0])
for i in newlist:
    if i not in symboles:
        newlist.remove(i)
print("len",len(newlist))
print("len",len(symboles))

len 388
len 1985


# Définition des paramètres

**Producer** permet d'envoyer des données tandis que **consumer** permet de lire en direct.

Le client de mongo permet de stocker ces valeurs dans notre database.

In [3]:

conf = {'bootstrap.servers': "host1:9092,host2:9092",
        'client.id': socket.gethostname()}

producerconnect = Producer({'bootstrap.servers': 'localhost:9092'})
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         api_version=(0,10,2))
consumer = KafkaConsumer(
    'ApiBinance',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     auto_commit_interval_ms=300,
     enable_auto_commit=True,
     group_id='my-group')

client = MongoClient("localhost", 27017)

In [6]:
#producer.send('ApiBinance',b'hesdvfsvbllo, Kafka')

In [10]:
db = client.Binance
collection = db.symbol
db.symbol.delete_many({})

In [97]:
# nom={"ID": 1, "symbol": "ETHBTC", "priceChangePercent": 0.654, "lastPrice": 0.07081200, "lastQty": 1.17340000, "highPrice": 0.07180400, "lowPrice": 0.07014000, "volume": 89116.07380000, "currentaverageprice": 0.07077283}
# nom2={"ID": 2, "symbol": "BTCUSDT", "priceChangePercent": 2.976, "lastPrice": 42512.00000000, "lastQty": 0.17164000, "highPrice": 43361.00000000, "lowPrice": 40738.48000000, "volume": 59150.16141000, "currentaverageprice": 42498.38531030}
# rec_id1 = collection.insert_one(nom)
# rec_id2 = collection.insert_one(nom2)

In [11]:
listeInsertion=[]
compteur=0
with open("fichier.txt", "r") as fichier:
	
	lines = fichier.readlines()
	for line in lines:
		listeInsertion.append(line)


print(len(listeInsertion))
list1=[]
list1=listeInsertion[0].split(',WEEE')
print(len(list1))

del list1[-1]

jsone=[]
print(list1[0][:len(i)-1])
for i in list1:
	try:
		jsone.append(json.loads(i))
		c=collection.insert_one(json.loads(i))
	except ValueError:
		print("error for",i,ValueError)


1
107
{"ID": 1, "symbol": "ETHBTC", "priceChangePercent": -0.068, "lastPrice": 0.07038700, "lastQty": 1.00410000, "highPrice": 0.07180400, "lowPrice": 0.07014000, "volume": 89625.31950000, "currentaverageprice": 0.07041493


# Insertion

On a récupérer les données du fichier. On utilise Kafka Connect pour faire le lien entre mongodb et les topics.

In [ ]:
cursor = collection.find()
for record in cursor:
    print('yo',record)

In [17]:
print(len(jsone))
print(jsone[0])
print(type(jsone[0]))
print(str(jsone[0]))
for i in jsone:
    producer.send('ApiBinance',bytes(str(i), encoding='utf-8'))
    sleep(0.5)
compteur=0


106
{'ID': 1, 'symbol': 'ETHBTC', 'priceChangePercent': -0.068, 'lastPrice': 0.070387, 'lastQty': 1.0041, 'highPrice': 0.071804, 'lowPrice': 0.07014, 'volume': 89625.3195, 'currentaverageprice': 0.07041493}
<class 'dict'>
{'ID': 1, 'symbol': 'ETHBTC', 'priceChangePercent': -0.068, 'lastPrice': 0.070387, 'lastQty': 1.0041, 'highPrice': 0.071804, 'lowPrice': 0.07014, 'volume': 89625.3195, 'currentaverageprice': 0.07041493}


On a inséré les données dans kafka topics. Maintenant, via Kafka connect on récupére les données puis on les met dans mongodb

**Kafka Connect**

Lire des données :

In [ ]:
from confluent_kafka import Consumer, KafkaError

settings = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'mygroup2',
    'client.id': 'client-2',
    'enable.auto.commit': True,
    'session.timeout.ms': 6000,
    'default.topic.config': {'auto.offset.reset': 'smallest'}
}

c = Consumer(settings)

c.subscribe(['ApiBinance'])

try:
    while True:
        msg = c.poll(0.1)
        if msg is None:
            continue
        elif not msg.error():
            print('Received message: {0}'.format(msg.value()))
        elif msg.error().code() == KafkaError._PARTITION_EOF:
            print('End of partition reached {0}/{1}'
                  .format(msg.topic(), msg.partition()))
        else:
            print('Error occured: {0}'.format(msg.error().str()))

except KeyboardInterrupt:
    pass

finally:
    c.close()

On les met dans MongoDB.

In [ ]:
def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: {0}: {1}"
              .format(msg.value(), err.str()))
    else:
        print("Message produced: {0}".format(msg.value()))

p = Producer({'bootstrap.servers': 'localhost:9092'})

try:
    for val in range(1, 1000):
        p.produce('ApiBinance', 'myvalue #{0}'
                  .format(val), callback=acked)
        p.poll(0.5)

except KeyboardInterrupt:
    pass

p.flush(30)

On fait des requetes

In [ ]:
for i in listeInsertion:
    producer.send('ApiBinance',bytes(i, encoding='utf-8'))
    sleep(0.5)
compteur=0
for message in consumer:
    compteur+=1
    print(compteur)
    if "symbol" in str(message.value,'utf-8'):
        print("yo",json.loads(str(message.value,'utf-8')))
        collection.insert_one(json.loads(str(message.value,'utf-8')))
        
